# 클래스4개(기타괴롭힘, 갈취, 직장내 괴롭힘, 협박)

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
!pip install konlpy

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# 한글 지원 폰트
plt.rc('font', family='NanumBarunGothic')

# 마이너스 부호 
plt.rcParams['axes.unicode_minus'] = False
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Mecab
from konlpy.tag import Okt

In [5]:
data = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_spell_check.csv')
data.head()

,class,conversation,labelling,label_sentence,cleaned_sentence,spell_checked_conversation
0,직장 내 괴롭힘 대화,길동씨 이번에 이것좀 처리해요\n이거 제가 한게 아닌데요\n팀에서 내가 니가가 어딨...,"1,2,1,2,1,2,1,2,1,2",1:길동씨 이번에 이것좀 처리해요\n2:이거 제가 한게 아닌데요\n1:팀에서 내가 ...,길동씨 이번에 이것좀 처리해요\n이거 제가 한게 아닌데요\n팀에서 내가 니가가 어딨...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...
1,기타 괴롭힘 대화,야 너 앞니 왜 그렇게 튀어나왔냐?\n 태어날때부터 그랬어 물어보지마.\n 아 그럼...,"1,2,1,2,1,2,1,2,1,1,1,2,1,1",1:야 너 앞니 왜 그렇게 튀어나왔냐?\n2:태어날때부터 그랬어 물어보지마.\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날때부터 그랬어 물어보지마.\n아 그럼 태...,야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날 때부터 그랬어 물어보지 마.\n아 그럼...
2,갈취 대화,원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n 미안 나 오늘은 진짜 돈이 없...,"1,2,1,2,1,2,1,2,1,2",1:원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n2:미안 나 오늘은 진짜 돈...,원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n미안 나 오늘은 진짜 돈이 없어...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼?\n미안 나 오늘은 진짜 돈이...
3,갈취 대화,너 저번에 술먹은 날 기억해?\n아니 왜?\n야 300만원만 가져와.\n뭔 소리야....,"1,2,1,2,1,2,1,2,1,2",1:너 저번에 술먹은 날 기억해?\n2:아니 왜?\n1:야 300만원만 가져와.\n...,너 저번에 술먹은 날 기억해?\n아니 왜?\n야 300만원만 가져와.\n뭔 소리야....,너 저번에 술 먹은 날 기억해?\n아니 왜?\n야 300만 원만 가져와.\n뭔 소리...
4,협박 대화,너 또 내말 무시하냐?\n 이새끼 널 좆으로 보나본데?\n아냐 진짜 시간이 없었어 ...,"1,1,2,1,1,2,1,2,1,1",1:너 또 내말 무시하냐?\n1:이새끼 널 좆으로 보나본데?\n2:아냐 진짜 시간이...,너 또 내말 무시하냐?\n이새끼 널 좆으로 보나본데?\n아냐 진짜 시간이 없었어 미...,너 또 내말 무시하냐?\n이 새끼 널 좆으로 보나 본데?\n아냐 진짜 시간이 없었어...


In [6]:
tokenizer = Okt()

In [7]:
data['class'].value_counts()

기타 괴롭힘 대화      1065
갈취 대화           978
직장 내 괴롭힘 대화     975
협박 대화           895
Name: class, dtype: int64

In [8]:
stop = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/한글불용어.csv')
stop.head()

,불용어
0,아
1,휴
2,아이구
3,아이쿠
4,아이고


In [9]:
kor_stop = []
for s in stop.불용어:
    kor_stop.append(s)

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

for s in stopwords:
    kor_stop.append(s)
    
kor_stop[:5]

['아', '휴', '아이구', '아이쿠', '아이고']

In [14]:
def read_token(Class):
    result = []
    for c in Class:
        tokenlist = tokenizer.pos(c, stem=True, norm=True)
        for word in tokenlist:
            if word[1] in ["Noun"]:
                result.append(word[0])

    return ' '.join(result)

In [45]:
etc = data[data['class']=='기타 괴롭힘 대화'].reset_index(drop=True).spell_checked_conversation

In [46]:
etc_list = read_token(etc)
len(etc_list)

102031

In [47]:
extort = data[data['class']=='갈취 대화'].reset_index(drop=True).spell_checked_conversation
extort_list = read_token(extort)
len(extort_list)

94354

In [48]:
company = data[data['class']=='직장 내 괴롭힘 대화'].reset_index(drop=True).spell_checked_conversation
company_list = read_token(company)
len(company_list)

106657

In [49]:
blackmaile = data[data['class']=='협박 대화'].reset_index(drop=True).spell_checked_conversation
blackmaile_list = read_token(blackmaile)
len(blackmaile_list)

98142

In [50]:
Class = []
Class.append(etc_list)
Class.append(extort_list)
Class.append(company_list)
Class.append(blackmaile_list)
len(Class)

4

In [51]:
# TF-IDF행렬(X) 생성
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(Class)
print(X.shape)

(4, 7007)


In [53]:
class_name = ['기타 괴롭힘 대화', '갈취 대화', '직장 내 괴롭힘 대화', '협박 대화']

m1 = X[0].tocoo()
m2 = X[1].tocoo() 
m3 = X[2].tocoo()
m4 = X[3].tocoo()

w1 = [[i, j] for i, j in zip(m1.col, m1.data)] 
w2 = [[i, j] for i, j in zip(m2.col, m2.data)]
w3 = [[i, j] for i, j in zip(m3.col, m2.data)]
w4 = [[i, j] for i, j in zip(m4.col, m2.data)]

w1.sort(key=lambda x: x[1], reverse=True) 
w2.sort(key=lambda x: x[1], reverse=True)
w3.sort(key=lambda x: x[1], reverse=True)
w4.sort(key=lambda x: x[1], reverse=True)

In [54]:
print("기타괴롭힘을 대표하는 단어들:")
for i in range(30):
    print(vectorizer.get_feature_names()[w1[i][0]], end=', ')

print('\n')

print('갈취 대화을 대표하는 단어들:')
for i in range(30):
    print(vectorizer.get_feature_names()[w2[i][0]], end=', ')

print('\n')

print('직장 내 괴롭힘을 대표하는 단어들:')
for i in range(30):
    print(vectorizer.get_feature_names()[w3[i][0]], end=', ')

print('\n')

print('협박을 대표하는 단어들:')
for i in range(30):
    print(vectorizer.get_feature_names()[w4[i][0]], end=', ')

기타괴롭힘을 대표하는 단어들:
진짜, 우리, 그냥, 무슨, 지금, 사람, 여기, 새끼, 엄마, 고객, 친구, 제발, 환불, 냄새, 손님, 얼굴, 거지, 생각, 장애인, 오늘, 기분, 돼지, 그게, 정말, 저기, 소리, 어디, 하나, 시간, 그거, 

갈취 대화을 대표하는 단어들:
진짜, 지금, 그냥, 제발, 무슨, 여기, 우리, 이번, 오늘, 엄마, 친구, 원만, 정말, 사람, 그거, 새끼, 얼마, 저번, 그게, 생각, 내일, 신고, 달라, 거기, 저희, 하나, 당장, 이제, 가지, 당신, 

직장 내 괴롭힘을 대표하는 단어들:
이해해, 일이십, 세금, 고등학생, 다나, 민호, 은행, 순이, 리가, 불복종, 무릎, 보드, 건가, 애초, 밀리, 고작, 핑계, 별로, 자주, 연인, 글쎄, 자기, 기적, 대화, 인상, 모습, 다해, 살이, 일도, 상관, 

협박을 대표하는 단어들:
이야기, 할머니, 상호, 간다, 수학, 성적, 오줌, 저리, 자기, 선배, 애초, 구나, 신경, 생각, 냄새, 오늘, 비교, 누구, 그동안, 얼마, 화가, 억지로, 구토, 처리, 정신, 시간, 내면, 조심, 코로나, 벌써, 

In [55]:
n = 15
w1_, w2_, w3_, w4_ = [], [], [], []

for i in range(100):
    w1_.append(vectorizer.get_feature_names()[w1[i][0]])
    w2_.append(vectorizer.get_feature_names()[w2[i][0]])
    w3_.append(vectorizer.get_feature_names()[w3[i][0]])
    w4_.append(vectorizer.get_feature_names()[w4[i][0]])

target_etc, target_extort, target_company, target_blackmaile = [], [], [], []

for i in range(100):
    if (w1_[i] not in w2_ + w3_ + w4_) : target_etc.append(w1_[i])
    if len(target_etc) == n: break 

for i in range(100):
    if (w2_[i] not in w1_ + w3_ + w4_) : target_extort.append(w2_[i])
    if len(target_extort) == n: break

for i in range(100):
    if (w3_[i] not in w1_ + w2_ + w4_) : target_company.append(w3_[i])
    if len(target_company) == n: break

for i in range(100):
    if (w4_[i] not in w1_ + w2_ + w3_) : target_blackmaile.append(w4_[i])
    if len(target_blackmaile) == n: break

In [56]:
len(target_etc), len(target_extort), len(target_company), len(target_blackmaile)

(15, 15, 15, 15)

In [58]:
target = [target_etc, target_extort, target_company, target_blackmaile]

for i, c in enumerate(class_name):
    print(c+' :')
    print(target[i])
    print()

기타 괴롭힘 대화 :
['고객', '환불', '손님', '얼굴', '장애인', '기분', '돼지', '때문', '남자', '병신', '얘기', '너희', '머리', '자꾸', '사과']

갈취 대화 :
['원만', '저번', '달라', '지갑', '가방', '번만', '어이', '입금', '선물', '학원', '현금', '잠깐', '차비', '핸드폰', '경찰']

직장 내 괴롭힘 대화 :
['일이십', '세금', '고등학생', '다나', '민호', '은행', '순이', '리가', '불복종', '무릎', '보드', '건가', '밀리', '고작', '핑계']

협박 대화 :
['이야기', '할머니', '상호', '간다', '수학', '성적', '오줌', '선배', '구나', '신경', '비교', '화가', '억지로', '구토', '처리']



# 클래스2개(가해자, 피해자)

In [38]:
data.head()

,class,conversation,labelling,label_sentence,cleaned_sentence,spell_checked_conversation
0,직장 내 괴롭힘 대화,길동씨 이번에 이것좀 처리해요\n이거 제가 한게 아닌데요\n팀에서 내가 니가가 어딨...,"1,2,1,2,1,2,1,2,1,2",1:길동씨 이번에 이것좀 처리해요\n2:이거 제가 한게 아닌데요\n1:팀에서 내가 ...,길동씨 이번에 이것좀 처리해요\n이거 제가 한게 아닌데요\n팀에서 내가 니가가 어딨...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...
1,기타 괴롭힘 대화,야 너 앞니 왜 그렇게 튀어나왔냐?\n 태어날때부터 그랬어 물어보지마.\n 아 그럼...,"1,2,1,2,1,2,1,2,1,1,1,2,1,1",1:야 너 앞니 왜 그렇게 튀어나왔냐?\n2:태어날때부터 그랬어 물어보지마.\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날때부터 그랬어 물어보지마.\n아 그럼 태...,야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날 때부터 그랬어 물어보지 마.\n아 그럼...
2,갈취 대화,원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n 미안 나 오늘은 진짜 돈이 없...,"1,2,1,2,1,2,1,2,1,2",1:원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n2:미안 나 오늘은 진짜 돈...,원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n미안 나 오늘은 진짜 돈이 없어...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼?\n미안 나 오늘은 진짜 돈이...
3,갈취 대화,너 저번에 술먹은 날 기억해?\n아니 왜?\n야 300만원만 가져와.\n뭔 소리야....,"1,2,1,2,1,2,1,2,1,2",1:너 저번에 술먹은 날 기억해?\n2:아니 왜?\n1:야 300만원만 가져와.\n...,너 저번에 술먹은 날 기억해?\n아니 왜?\n야 300만원만 가져와.\n뭔 소리야....,너 저번에 술 먹은 날 기억해?\n아니 왜?\n야 300만 원만 가져와.\n뭔 소리...
4,협박 대화,너 또 내말 무시하냐?\n 이새끼 널 좆으로 보나본데?\n아냐 진짜 시간이 없었어 ...,"1,1,2,1,1,2,1,2,1,1",1:너 또 내말 무시하냐?\n1:이새끼 널 좆으로 보나본데?\n2:아냐 진짜 시간이...,너 또 내말 무시하냐?\n이새끼 널 좆으로 보나본데?\n아냐 진짜 시간이 없었어 미...,너 또 내말 무시하냐?\n이 새끼 널 좆으로 보나 본데?\n아냐 진짜 시간이 없었어...


In [80]:
sen = data['label_sentence']
attack = []
damaged = []
for i in range(3912):
    sen_tokens = sen[i].split('\n')
    for s in sen_tokens:
        if s[0] == str(1):
            attack.append(s[2:])
        else:
            damaged.append(s[2:])

In [83]:
len(attack), len(damaged)

(21285, 19143)

In [87]:
attack_list = read_token(attack)

In [88]:
len(attack_list)

248762

In [89]:
damaged_list = read_token(damaged)
len(damaged_list)

126297

In [90]:
Class = []
Class.append(attack_list)
Class.append(damaged_list)
len(Class)

2

In [91]:
# TF-IDF행렬(X) 생성
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(Class)
print(X.shape)

(2, 7626)


In [92]:
class_name = ['가해자', '피해자']

In [93]:
m1 = X[0].tocoo()
m2 = X[1].tocoo() 

w1 = [[i, j] for i, j in zip(m1.col, m1.data)] 
w2 = [[i, j] for i, j in zip(m2.col, m2.data)]

w1.sort(key=lambda x: x[1], reverse=True) 
w2.sort(key=lambda x: x[1], reverse=True)

In [94]:
print("가해자을 대표하는 단어들:")
for i in range(30):
    print(vectorizer.get_feature_names()[w1[i][0]], end=', ')

print('\n')

print('피해자을 대표하는 단어들:')
for i in range(30):
    print(vectorizer.get_feature_names()[w2[i][0]], end=', ')

가해자을 대표하는 단어들:
지금, 진짜, 우리, 그냥, 사람, 새끼, 오늘, 하나, 여기, 회사, 생각, 친구, 대리, 이번, 무슨, 자네, 어디, 당장, 내일, 엄마, 한번, 그거, 얼굴, 당신, 시간, 이제, 소리, 누가, 장난, 저기, 

피해자을 대표하는 단어들:
제발, 진짜, 무슨, 지금, 정말, 그게, 말씀, 우리, 부장, 사람, 시간, 여기, 오늘, 이번, 그냥, 엄마, 생각, 신고, 저희, 소리, 이제, 고객, 조금, 손님, 과장, 다시, 갑자기, 미안, 저번, 그거, 

In [95]:
n = 15
w1_, w2_ = [], []

for i in range(100):
    w1_.append(vectorizer.get_feature_names()[w1[i][0]])
    w2_.append(vectorizer.get_feature_names()[w2[i][0]])

target_attack, target_damaged = [], []

for i in range(100):
    if (w1_[i] not in w2_) : target_attack.append(w1_[i])
    if len(target_attack) == n: break 

for i in range(100):
    if (w2_[i] not in w1_) : target_damaged.append(w2_[i])
    if len(target_damaged) == n: break

In [96]:
len(target_attack), len(target_damaged)

(15, 15)

In [98]:
target = [target_attack, target_damaged]

for i, c in enumerate(class_name):
    print(c+' :')
    print(target[i])
    print()

가해자 :
['새끼', '자네', '얼굴', '누가', '거기', '요즘', '정신', '아주', '거지', '제대로', '어이', '머리', '학교', '가지', '냄새']

피해자 :
['제발', '말씀', '부장', '저희', '고객', '조금', '손님', '과장', '갑자기', '미안', '가요', '장님', '경찰', '러시', '협박']



# Verb 추가

In [107]:
def read_token(Class):
    result = []
    for c in Class:
        tokenlist = tokenizer.pos(c, stem=True, norm=True)
        for word in tokenlist:
            if word[1] in ["Noun", "Verb"]:
                result.append(word[0])

    return ' '.join(result)

In [108]:
etc = data[data['class']=='기타 괴롭힘 대화'].reset_index(drop=True).spell_checked_conversation
etc_list = read_token(etc)

extort = data[data['class']=='갈취 대화'].reset_index(drop=True).spell_checked_conversation
extort_list = read_token(extort)

company = data[data['class']=='직장 내 괴롭힘 대화'].reset_index(drop=True).spell_checked_conversation
company_list = read_token(company)

blackmaile = data[data['class']=='협박 대화'].reset_index(drop=True).spell_checked_conversation
blackmaile_list = read_token(blackmaile)

len(etc_list),len(extort_list), len(company_list), len(blackmaile_list)

(152343, 145550, 154566, 155464)

In [109]:
Class = []
Class.append(etc_list)
Class.append(extort_list)
Class.append(company_list)
Class.append(blackmaile_list)
len(Class)

4

In [110]:
# TF-IDF행렬(X) 생성
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(Class)
print(X.shape)

(4, 7886)


In [111]:
class_name = ['기타 괴롭힘 대화', '갈취 대화', '직장 내 괴롭힘 대화', '협박 대화']

m1 = X[0].tocoo()
m2 = X[1].tocoo() 
m3 = X[2].tocoo()
m4 = X[3].tocoo()

w1 = [[i, j] for i, j in zip(m1.col, m1.data)] 
w2 = [[i, j] for i, j in zip(m2.col, m2.data)]
w3 = [[i, j] for i, j in zip(m3.col, m2.data)]
w4 = [[i, j] for i, j in zip(m4.col, m2.data)]

w1.sort(key=lambda x: x[1], reverse=True) 
w2.sort(key=lambda x: x[1], reverse=True)
w3.sort(key=lambda x: x[1], reverse=True)
w4.sort(key=lambda x: x[1], reverse=True)

In [112]:
print("기타괴롭힘을 대표하는 단어들:")
for i in range(30):
    print(vectorizer.get_feature_names()[w1[i][0]], end=', ')

print('\n')

print('갈취 대화을 대표하는 단어들:')
for i in range(30):
    print(vectorizer.get_feature_names()[w2[i][0]], end=', ')

print('\n')

print('직장 내 괴롭힘을 대표하는 단어들:')
for i in range(30):
    print(vectorizer.get_feature_names()[w3[i][0]], end=', ')

print('\n')

print('협박을 대표하는 단어들:')
for i in range(30):
    print(vectorizer.get_feature_names()[w4[i][0]], end=', ')

기타괴롭힘을 대표하는 단어들:
하다, 보다, 진짜, 알다, 오다, 우리, 그냥, 무슨, 되다, 가다, 해주다, 지금, 맞다, 사람, 돼다, 먹다, 다니다, 자다, 나다, 여기, 새끼, 생기다, 들다, 주다, 엄마, 고객, 모르다, 친구, 제발, 나오다, 

갈취 대화을 대표하는 단어들:
하다, 보다, 돼다, 주다, 빌리다, 알다, 진짜, 되다, 내놓다, 지금, 오다, 맞다, 그냥, 나다, 제발, 가다, 무슨, 여기, 우리, 자다, 받다, 이번, 오늘, 엄마, 보내다, 싶다, 친구, 나오다, 돌려주다, 원만, 

직장 내 괴롭힘을 대표하는 단어들:
튀다, 지다, 거울, 미주, 기다, 넘다, 떨어지다, 이유, 박다, 되어다, 그게, 수치, 깨지다, 병신, 공부, 절대, 다리다, 다치다, 맡다, 잠시, 여행, 처지다, 흐흑, 원래, 먹기, 인성, 지나가다, 위치, 머리, 귀신, 

협박을 대표하는 단어들:
해먹, 초등학교, 요즘, 말씀, 계속, 죽다, 딸아이, 넘다, 싸다, 하루, 보내다, 일해, 가리다, 말투, 이야기, 지나치다, 떨어지다, 도대체, 평상시, 듣기, 때우다, 세금, 고작, 대박, 제대로, 고민, 계시, 어딨다, 시간, 겨우, 

In [114]:
n = 15
w1_, w2_, w3_, w4_ = [], [], [], []

for i in range(100):
    w1_.append(vectorizer.get_feature_names()[w1[i][0]])
    w2_.append(vectorizer.get_feature_names()[w2[i][0]])
    w3_.append(vectorizer.get_feature_names()[w3[i][0]])
    w4_.append(vectorizer.get_feature_names()[w4[i][0]])

target_etc, target_extort, target_company, target_blackmaile = [], [], [], []

for i in range(100):
    if (w1_[i] not in w2_ + w3_ + w4_) : target_etc.append(w1_[i])
    if len(target_etc) == n: break 

for i in range(100):
    if (w2_[i] not in w1_ + w3_ + w4_) : target_extort.append(w2_[i])
    if len(target_extort) == n: break

for i in range(100):
    if (w3_[i] not in w1_ + w2_ + w4_) : target_company.append(w3_[i])
    if len(target_company) == n: break

for i in range(100):
    if (w4_[i] not in w1_ + w2_ + w3_) : target_blackmaile.append(w4_[i])
    if len(target_blackmaile) == n: break

In [115]:
len(target_etc), len(target_extort), len(target_company), len(target_blackmaile)

(15, 15, 15, 15)

In [116]:
target = [target_etc, target_extort, target_company, target_blackmaile]

for i, c in enumerate(class_name):
    print(c+' :')
    print(target[i])
    print()

기타 괴롭힘 대화 :
['생기다', '고객', '환불', '손님', '얼굴', '장애인', '기분', '뭐라다', '찌다', '선생님', '보고', '부모님', '따다', '따르다', '놀리다']

갈취 대화 :
['빌리다', '내놓다', '돌려주다', '원만', '갚다', '가져오다', '얼마', '뒤지다', '저번', '내일', '달라', '거기', '가져가다', '당장', '버리다']

직장 내 괴롭힘 대화 :
['튀다', '지다', '미주', '기다', '이유', '박다', '되어다', '수치', '깨지다', '절대', '다리다', '다치다', '맡다', '잠시', '여행']

협박 대화 :
['해먹', '초등학교', '요즘', '계속', '딸아이', '싸다', '하루', '말투', '이야기', '지나치다', '도대체', '평상시', '듣기', '때우다', '세금']



# Verb추가 (가해자, 피해자)

In [117]:
sen = data['label_sentence']
attack = []
damaged = []
for i in range(3912):
    sen_tokens = sen[i].split('\n')
    for s in sen_tokens:
        if s[0] == str(1):
            attack.append(s[2:])
        else:
            damaged.append(s[2:])

In [118]:
attack_list = read_token(attack)
damaged_list = read_token(damaged)

In [119]:
Class = []
Class.append(attack_list)
Class.append(damaged_list)
len(Class)

2

In [120]:
# TF-IDF행렬(X) 생성
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(Class)
print(X.shape)

(2, 8520)


In [121]:
class_name = ['가해자', '피해자']

m1 = X[0].tocoo()
m2 = X[1].tocoo() 

w1 = [[i, j] for i, j in zip(m1.col, m1.data)] 
w2 = [[i, j] for i, j in zip(m2.col, m2.data)]

w1.sort(key=lambda x: x[1], reverse=True) 
w2.sort(key=lambda x: x[1], reverse=True)

In [122]:
print("가해자을 대표하는 단어들:")
for i in range(30):
    print(vectorizer.get_feature_names()[w1[i][0]], end=', ')

print('\n')

print('피해자을 대표하는 단어들:')
for i in range(30):
    print(vectorizer.get_feature_names()[w2[i][0]], end=', ')

가해자을 대표하는 단어들:
하다, 보다, 지금, 알다, 되다, 진짜, 죽다, 우리, 오다, 주다, 그냥, 사람, 가다, 맞다, 돼다, 버리다, 먹다, 새끼, 해주다, 오늘, 들다, 자다, 않다, 나오다, 빌리다, 모르다, 싶다, 다니다, 하나, 여기, 

피해자을 대표하는 단어들:
하다, 알다, 제발, 진짜, 돼다, 무슨, 지금, 보다, 정말, 되다, 말다, 그게, 말씀, 우리, 살리다, 이르다, 가다, 주다, 드리다, 않다, 부장, 해주다, 사람, 나다, 시간, 여기, 오늘, 오다, 모르다, 이번, 

In [123]:
n = 15
w1_, w2_ = [], []

for i in range(100):
    w1_.append(vectorizer.get_feature_names()[w1[i][0]])
    w2_.append(vectorizer.get_feature_names()[w2[i][0]])

target_attack, target_damaged = [], []

for i in range(100):
    if (w1_[i] not in w2_) : target_attack.append(w1_[i])
    if len(target_attack) == n: break 

for i in range(100):
    if (w2_[i] not in w1_) : target_damaged.append(w2_[i])
    if len(target_damaged) == n: break

In [124]:
target = [target_attack, target_damaged]

for i, c in enumerate(class_name):
    print(c+' :')
    print(target[i])
    print()

가해자 :
['버리다', '새끼', '다니다', '하나', '내놓다', '죽이다', '치다', '자네', '생기다', '당장', '얼굴', '살다', '누가', '따다', '뒤지다']

피해자 :
['제발', '정말', '말씀', '살리다', '드리다', '부장', '저희', '고객', '조금', '손님', '과장', '갑자기', '돌려주다', '미안', '저번']



# 토크나이저X, 띄어쓰기로 토큰화

In [135]:
import re

def read_token(Class):
    result = []
    for c in Class:
        c = re.sub('\n', ' ', c)
        tokenlist = c.split(' ')
        for word in tokenlist:
            result.append(word)

    return ' '.join(result)

In [136]:
etc = data[data['class']=='기타 괴롭힘 대화'].reset_index(drop=True).spell_checked_conversation
etc_list = read_token(etc)

extort = data[data['class']=='갈취 대화'].reset_index(drop=True).spell_checked_conversation
extort_list = read_token(extort)

company = data[data['class']=='직장 내 괴롭힘 대화'].reset_index(drop=True).spell_checked_conversation
company_list = read_token(company)

blackmaile = data[data['class']=='협박 대화'].reset_index(drop=True).spell_checked_conversation
blackmaile_list = read_token(blackmaile)

len(etc_list),len(extort_list), len(company_list), len(blackmaile_list)

(231751, 217247, 237804, 226287)

In [141]:
Class = []
Class.append(etc_list)
Class.append(extort_list)
Class.append(company_list)
Class.append(blackmaile_list)
len(Class)

4

In [140]:
# TF-IDF행렬(X) 생성
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(Class)
print(X.shape)

(4, 39600)


In [142]:
class_name = ['기타 괴롭힘 대화', '갈취 대화', '직장 내 괴롭힘 대화', '협박 대화']

m1 = X[0].tocoo()
m2 = X[1].tocoo() 
m3 = X[2].tocoo()
m4 = X[3].tocoo()

w1 = [[i, j] for i, j in zip(m1.col, m1.data)] 
w2 = [[i, j] for i, j in zip(m2.col, m2.data)]
w3 = [[i, j] for i, j in zip(m3.col, m2.data)]
w4 = [[i, j] for i, j in zip(m4.col, m2.data)]

w1.sort(key=lambda x: x[1], reverse=True) 
w2.sort(key=lambda x: x[1], reverse=True)
w3.sort(key=lambda x: x[1], reverse=True)
w4.sort(key=lambda x: x[1], reverse=True)

In [143]:
print("기타괴롭힘을 대표하는 단어들:")
for i in range(30):
    print(vectorizer.get_feature_names()[w1[i][0]], end=', ')

print('\n')

print('갈취 대화을 대표하는 단어들:')
for i in range(30):
    print(vectorizer.get_feature_names()[w2[i][0]], end=', ')

print('\n')

print('직장 내 괴롭힘을 대표하는 단어들:')
for i in range(30):
    print(vectorizer.get_feature_names()[w3[i][0]], end=', ')

print('\n')

print('협박을 대표하는 단어들:')
for i in range(30):
    print(vectorizer.get_feature_names()[w4[i][0]], end=', ')

기타괴롭힘을 대표하는 단어들:
내가, 진짜, 네가, 거야, 아니, 그냥, 그래, 무슨, 아니야, 그런, 그렇게, 우리, 지금, 하지, 죄송합니다, 너무, 그럼, 이거, 고객님, 어떻게, 여기, 제발, 그만해, 제가, 이렇게, 하는, 같은, 말이, 너네, 손님, 

갈취 대화을 대표하는 단어들:
내가, 진짜, 그럼, 이거, 아니, 지금, 그래, 거야, 없어, 돈이, 그냥, 내놔, 없어요, 무슨, 안돼, 제발, 네가, 빨리, 제가, 나도, 원만, 그건, 우리, 있어, 있는, 어떻게, 정말, 오늘, 여기, 빌려줘, 

직장 내 괴롭힘을 대표하는 단어들:
처리, 약속, 사장님이, 넘어, 카페에, 있거든, 그럼요, 너흰, 해줄까, 이름은, 먹으면서, 죄송한, 오세요, 관심이지, 미안해요, 해주면, 거잖아요, 괜찮습니다, 맞아요, 꼬시려고, 창피해서, 알겠냐, 버는, 보여줄게, 어울려, 말하기만, 그런다고, 있어야지, 하는지, 햄버거, 

협박을 대표하는 단어들:
아니긴, 내놔, 나네, 있는지, 한마디, 넘어, 가져가, 웃긴다, 이만, 걸지, 거짓말하는, 준비하고, 해주면, 말씀, 기저귀, 시비, 돼요, 우습게, 피해, 맞나요, 번이나, 한국에서, 능력, 싫어요, 아니고요, 사람으로, 할머니, 구경하고, 싫다고, 죄를, 

In [144]:
n = 15
w1_, w2_, w3_, w4_ = [], [], [], []

for i in range(100):
    w1_.append(vectorizer.get_feature_names()[w1[i][0]])
    w2_.append(vectorizer.get_feature_names()[w2[i][0]])
    w3_.append(vectorizer.get_feature_names()[w3[i][0]])
    w4_.append(vectorizer.get_feature_names()[w4[i][0]])

target_etc, target_extort, target_company, target_blackmaile = [], [], [], []

for i in range(100):
    if (w1_[i] not in w2_ + w3_ + w4_) : target_etc.append(w1_[i])
    if len(target_etc) == n: break 

for i in range(100):
    if (w2_[i] not in w1_ + w3_ + w4_) : target_extort.append(w2_[i])
    if len(target_extort) == n: break

for i in range(100):
    if (w3_[i] not in w1_ + w2_ + w4_) : target_company.append(w3_[i])
    if len(target_company) == n: break

for i in range(100):
    if (w4_[i] not in w1_ + w2_ + w3_) : target_blackmaile.append(w4_[i])
    if len(target_blackmaile) == n: break

In [145]:
target = [target_etc, target_extort, target_company, target_blackmaile]

for i, c in enumerate(class_name):
    print(c+' :')
    print(target[i])
    print()

기타 괴롭힘 대화 :
['하지', '고객님', '그만해', '하는', '같은', '말이', '손님', '너는', '그만', '기분', '뭐가', '나는', '얘들아', '거지', '것도']

갈취 대화 :
['돈이', '없어요', '안돼', '원만', '빌려줘', '좋은', '안돼요', '알겠어', '줄래', '빌려주라', '돈을', '얼마', '뒤져서', '거기', '이건']

직장 내 괴롭힘 대화 :
['처리', '약속', '사장님이', '있거든', '너흰', '해줄까', '이름은', '먹으면서', '죄송한', '오세요', '관심이지', '미안해요', '거잖아요', '괜찮습니다', '맞아요']

협박 대화 :
['아니긴', '나네', '있는지', '한마디', '가져가', '웃긴다', '이만', '걸지', '거짓말하는', '준비하고', '말씀', '기저귀', '시비', '돼요', '우습게']



# 띄어쓰기(가해자,피해자)

In [146]:
import re

def read_token(Class):
    result = []
    for c in Class:
        c = re.sub('\n', ' ', c)
        tokenlist = c.split(' ')
        for word in tokenlist:
            result.append(word)

    return ' '.join(result)

In [147]:
attack_list = read_token(attack)
damaged_list = read_token(damaged)

In [148]:
Class = []
Class.append(attack_list)
Class.append(damaged_list)
len(Class)

2

In [149]:
# TF-IDF행렬(X) 생성
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(Class)
print(X.shape)

(2, 51013)


In [150]:
class_name = ['가해자', '피해자']

m1 = X[0].tocoo()
m2 = X[1].tocoo() 

w1 = [[i, j] for i, j in zip(m1.col, m1.data)] 
w2 = [[i, j] for i, j in zip(m2.col, m2.data)]

w1.sort(key=lambda x: x[1], reverse=True) 
w2.sort(key=lambda x: x[1], reverse=True)

In [151]:
print("가해자을 대표하는 단어들:")
for i in range(30):
    print(vectorizer.get_feature_names()[w1[i][0]], end=', ')

print('\n')

print('피해자을 대표하는 단어들:')
for i in range(30):
    print(vectorizer.get_feature_names()[w2[i][0]], end=', ')

가해자을 대표하는 단어들:
내가, 그럼, 지금, 아니, 그냥, 진짜, 그래, 니가, 이거, 그렇게, 우리, 오늘, 빨리, 너가, 어떻게, 이렇게, 있어, 무슨, 너무, 없어, 여기, 아니야, 나도, 알아, 당장, 이게, 근데, 내놔, 나한테, 같이, 

피해자을 대표하는 단어들:
죄송합니다, 제발, 제가, 내가, 진짜, 아니, 무슨, 지금, 정말, 아니야, 그게, 그건, 미안해, 이거, 아닙니다, 너무, 알겠습니다, 안돼, 없어요, 어떻게, 그래, 우리, 그렇게, 없어, 그래도, 부장님, 그냥, 그런, 나도, 그럼, 

In [152]:
n = 15
w1_, w2_ = [], []

for i in range(100):
    w1_.append(vectorizer.get_feature_names()[w1[i][0]])
    w2_.append(vectorizer.get_feature_names()[w2[i][0]])

target_attack, target_damaged = [], []

for i in range(100):
    if (w1_[i] not in w2_) : target_attack.append(w1_[i])
    if len(target_attack) == n: break 

for i in range(100):
    if (w2_[i] not in w1_) : target_damaged.append(w2_[i])
    if len(target_damaged) == n: break

target = [target_etc, target_extort, target_company, target_blackmaile]

for i, c in enumerate(class_name):
    print(c+' :')
    print(target[i])
    print()

가해자 :
['하지', '고객님', '그만해', '하는', '같은', '말이', '손님', '너는', '그만', '기분', '뭐가', '나는', '얘들아', '거지', '것도']

피해자 :
['돈이', '없어요', '안돼', '원만', '빌려줘', '좋은', '안돼요', '알겠어', '줄래', '빌려주라', '돈을', '얼마', '뒤져서', '거기', '이건']

